In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
pd.set_option('display.max_rows',1000)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,ShuffleSplit,GridSearchCV,StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from math import sqrt
import lightgbm as lgb
import xgboost as xgb
from tqdm import tqdm
from rfpimp import *
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This wasnt you average time series competition, and i kinda didnt treat it as one it was more of a sequence prediction one in certain groups, basically if you can tell the model how is the sequence like and include more pattern recognition features you could improve scores,
this was fun if you figure out the pattern which i did and i loved it as i added more features and it boosted my cv

So, Since this competition was a repeat of a previous comp,I still tried my best to get the best scores 

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')


In [ ]:
final = pd.concat([df_train,df_test])

This Basic FE gets me to 84.7xx

In [ ]:
final['Avg_Dose_Insect'] = final['Estimated_Insects_Count']/final['Number_Doses_Week']
final['TotalDoses'] = final['Number_Doses_Week'] * final['Number_Weeks_Used']
# final['Doses_Week_Quit'] = final['Number_Doses_Week'] + final['Number_Weeks_Quit']
final['Doses_Week_Used_Quit'] = final['Number_Weeks_Used'] + final['Number_Weeks_Quit']
final['Doses_Week_Used_Quit_Divide'] = final['Number_Weeks_Used'] / final['Number_Weeks_Quit']

But after seeing LB just 3 hrs after the comp started i saw people getting 96.xx so i thought well i should see some eda and after seeing eda i still couldnt figure out anything,so i just saw the data and saw that the insect data was in a sequence of some sorts, so I did create groups based on increasing order of insects, then after that my lb increased to 88, then i tried to see more such Patterns and then i found the same patter in the Doses Column

In [ ]:
final = final.sort_values(by='ID')
final.reset_index(inplace=True,drop=True)

Creating Some Groups

In [ ]:
from collections import defaultdict
dict_groups = defaultdict()

The below code will create groups/clusters on the basis of IDs so later you can join it properly

In [ ]:
res = [[]]
counter = 0
for i, j in zip(final['Number_Doses_Week'].tolist(),final['Number_Doses_Week'].tolist()[1:]):
    if i <= j or i-2<=j:
        res[-1].append(final.iloc[counter]['ID'])
        counter += 1
    else:
        res.append([final.iloc[counter]['ID']]) 
        counter += 1
#There was a bug in the above code so it doesn't take ,last ids so manually append for doses and 
res[21640].append('F00155945')

   The Above loop had some bugs so i had to create a function to remove those bugs, you can optimize it the way you want

In [ ]:
def get_proper_groups(res):
    for i in range(len(res)):
        if i == 0:
            continue
        else:
            res[i-1].append(res[i][0])
            res[i].remove(res[i][0])
    dict_groups = defaultdict()
    for i in range(len(res)):
        dict_groups['group_{}'.format(i)] = res[i]
    return dict_groups

In [ ]:
final_groups = get_proper_groups(res)
#The above function is also there to correct some bugs in the code to get the groups

Since , creating the dictionary of those groups take time,i suggest just save it as a pickle

In [ ]:
import pickle
with open('groups_data_doses.pkl','rb') as f:
    groups_doses = pickle.load(f)
import pickle
with open('groups_data.pkl','rb') as f:
    final_groups = pickle.load(f)
#Load the pickled files

Converting dictionaries to dataframe,the merging with original data, now we have our two groups

In [ ]:
df_groups = pd.DataFrame(final_groups.items(),columns=['Groups','ID'])
df_doses =pd.DataFrame(groups_doses.items(),columns=['Groups_Doses','ID'])

In [ ]:
s = df_groups.apply(lambda x:pd.Series(x['ID']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'ID'
df_groups = df_groups.drop('ID',axis=1).join(s)
s = df_doses.apply(lambda x:pd.Series(x['ID']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'ID'
df_doses = df_doses.drop('ID',axis=1).join(s)
final = pd.merge(final,df_groups,on='ID')
final = pd.merge(final,df_doses,on='ID')
#This where I merge the groups

Creating some features on groups,i.e basically when does a group start or end etc, ie when does a cluster start or a cluster end

In [ ]:
def get_start_cycle(x):
    x = x.tolist()
    return x[0]
def get_end_cycle(x):
    x = x.tolist()
    return x[-1]

In [ ]:
temp = final.groupby(['Groups'])['ID'].agg(get_start_cycle)
temp = temp.reset_index()
temp['Start_Sequence_Insect'] = 1
final = pd.merge(final,temp[['ID','Start_Sequence_Insect']],on='ID',how='left')
temp = final.groupby(['Groups'])['ID'].agg(get_end_cycle)
temp = temp.reset_index()
temp['End_Sequence_Insect'] = 1
final = pd.merge(final,temp[['ID','End_Sequence_Insect']],on='ID',how='left')
temp = final.groupby(['Groups_Doses'])['ID'].agg(get_start_cycle)
temp = temp.reset_index()
temp['Start_Sequence_Doses'] = 1
final = pd.merge(final,temp[['ID','Start_Sequence_Doses']],on='ID',how='left')
temp = final.groupby(['Groups_Doses'])['ID'].agg(get_end_cycle)
temp = temp.reset_index()
temp['End_Sequence_Insect_Doses'] = 1
final = pd.merge(final,temp[['ID','End_Sequence_Insect_Doses']],on='ID',how='left')

In [ ]:
final['Start_Sequence_Insect'].fillna(0,inplace = True)
final['End_Sequence_Insect'].fillna(0,inplace = True)
final['Start_Sequence_Doses'].fillna(0,inplace = True)
final['End_Sequence_Insect_Doses'].fillna(0,inplace = True)

The below funtions help me to get features like,when and if the soil type,crop types changes after certain group,i create a binary feature on the basis of that

In [ ]:
def type_change_groups(x,groups):
    list1 = []
    for i in tqdm(range(len(final))):
        if i != final.shape[0]-1 and final.iloc[i+1][x] != final.iloc[i][x] and final.iloc[i+1][groups] != final.iloc[i][groups]:
            list1.append(1)
        else:
            list1.append(0)
    return list1

Some stats based FEATURES

In [ ]:
temp = final.groupby(['ID','Groups']).agg({'ID':['count'],
                                   'Number_Weeks_Quit':['sum','min','max','mean','std'],
                                   'Number_Weeks_Used':['sum','min','max','mean','std'],
                                           'Number_Doses_Week':['sum','min','max','mean','std'],
                                   'TotalDoses':['sum','min','max','mean','std'],
                                    'Doses_Week_Used_Quit':['sum','min','max','mean','std'],})
temp.columns = ['_ID_GROUP_COUNT_'.join(x) for x in temp.columns]
final = pd.merge(final,temp,on=['ID','Groups'])

In [ ]:
temp = final.groupby(['ID','Groups_Doses']).agg({'ID':['count'],
                                   'Number_Weeks_Quit':['sum','min','max','mean','std'],
                                   'Number_Weeks_Used':['sum','min','max','mean','std'],
                                           'Number_Doses_Week':['sum','min','max','mean','std'],
                                   'TotalDoses':['sum','min','max','mean','std'],
                                    'Doses_Week_Used_Quit':['sum','min','max','mean','std'],})
temp.columns = ['_ID_Doses_COUNT_'.join(x) for x in temp.columns]
final = pd.merge(final,temp,on=['ID','Groups_Doses'])

In [ ]:
temp = final.groupby(['Groups']).agg({'ID':['count'],})
temp.columns = ['_ONLY_GROUP_COUNT_'.join(x) for x in temp.columns]
final = pd.merge(final,temp,on=['Groups'])

In [ ]:
temp = final.groupby(['Groups_Doses']).agg({'ID':['count']})
temp.columns = ['_'.join(x) for x in temp.columns]
final = pd.merge(final,temp,on=['Groups_Doses'])

Below features are the most exciting ones as they tell you,whats the next crop damage in a certain group or a previous damage in a certain group its quite intutive , basically in a time series you it helps if you can sort of get an idea about that,or a next insect in a certain group this boosted my cv a lot

The first features , are trying to proabably guess a sequence of what will be the next crop_damage in a certain group

In [ ]:
for i in tqdm(range(1,10)):
    final[f'next_{i}_crop'] = final.groupby(['Groups'])['Crop_Damage'].apply(lambda x: x.shift(i).ffill())
    final[f'previous_{i}_crop'] = final.groupby(['Groups'])['Crop_Damage'].apply(lambda x: x.shift(-i).bfill())
    final[f'next_{i}_crop_Doses'] = final.groupby(['Groups_Doses'])['Crop_Damage'].apply(lambda x: x.shift(i).ffill())
    final[f'previous_{i}_crop_Doses'] = final.groupby(['Groups_Doses'])['Crop_Damage'].apply(lambda x: x.shift(-i).bfill())
    final[f'next_{i}_insects'] = final.groupby(['Groups'])['Estimated_Insects_Count'].apply(lambda x: x.shift(i).ffill())
#     final[f'next_{i}_insects_dose'] = final.groupby(['Groups'])['Number_Doses_Week'].apply(lambda x: x.shift(i).ffill())

In [ ]:
for i in tqdm(range(1,10)):
    final[f'next_{i}_Dose'] = final.groupby(['Groups_Doses'])['Number_Doses_Week'].apply(lambda x: x.shift(i).ffill())
#     final[f'previous_{i}_Dose_insect'] = final.groupby(['Groups_Doses'])['Estimated_Insects_Count'].apply(lambda x: x.shift(i).ffill())
    

this is doing the same thing as above , but in a very different way i.e by taking and group, and the characteristics of that group,now this didnt help that much,
but was the part of the FE 

In [ ]:
for i in tqdm(range(1,10)):
    final[f'next_{i}_Soil'] = final.groupby(['Groups','Soil_Type'])['Crop_Damage'].apply(lambda x: x.shift(i).ffill())
    final[f'next_{i}_Crop'] = final.groupby(['Groups','Crop_Type'])['Crop_Damage'].apply(lambda x: x.shift(i).ffill())
    final[f'next_{i}_Pesticide'] = final.groupby(['Groups','Pesticide_Use_Category'])['Crop_Damage'].apply(lambda x: x.shift(i).ffill())
    final[f'next_{i}_Season'] = final.groupby(['Groups','Season'])['Crop_Damage'].apply(lambda x: x.shift(i).ffill())
    final[f'next_{i}_Soil_D'] = final.groupby(['Groups_Doses','Soil_Type'])['Crop_Damage'].apply(lambda x: x.shift(i).ffill())
    final[f'next_{i}_Crop_D'] = final.groupby(['Groups_Doses','Crop_Type'])['Crop_Damage'].apply(lambda x: x.shift(i).ffill())
    final[f'next_{i}_Pesticide_D'] = final.groupby(['Groups_Doses','Pesticide_Use_Category'])['Crop_Damage'].apply(lambda x: x.shift(i).ffill())
    final[f'next_{i}_Season_D'] = final.groupby(['Groups_Doses','Season'])['Crop_Damage'].apply(lambda x: x.shift(i).ffill())
    
#     final[f'previous_{i}_Dose_insect'] = final.groupby(['Groups_Doses'])['Estimated_Insects_Count'].apply(lambda x: x.shift(i).ffill())
    

This is basically the function above i told you

In [ ]:
final['Soil_groups']=type_change_groups(x='Soil_Type',groups='Groups')
final['Crop_groups']=type_change_groups(x='Crop_Type',groups='Groups')
final['Season_groups']=type_change_groups(x='Season',groups='Groups')
final['Pesticide_groups']=type_change_groups(x='Pesticide_Use_Category',groups='Groups')

More FE

In [ ]:
final['Seq_Add_Doses'] = final['Start_Sequence_Doses'] + final['End_Sequence_Insect_Doses']
final['Seq_Add_Insect'] = final['Start_Sequence_Insect'] + final['End_Sequence_Insect']

In [ ]:
final['Groups'] = pd.factorize(final.Groups)[0]
final['Groups_Doses'] = pd.factorize(final.Groups_Doses)[0]

In [ ]:
df_train = final[final['Crop_Damage'].notnull()]
df_test = final[final['Crop_Damage'].isnull()]

In [ ]:
df_test.drop(['Crop_Damage'],axis=1,inplace=True)

Filling nulls

In [ ]:
df_train.fillna(-9999,inplace=True)
df_test.fillna(-9999,inplace=True)

The below function does the expanded mean encoding simply of the group variable seems to increase my cv,now this meamy n encoding is a bit irregular but is the most stable of all it is also used by catboost internally, this function like my notebook is a bit poorly coded (I m working on my coding a bit)

In [ ]:
def expanding_mean(df1,df2,df_test=None,target=None,cols=None):
    df_1 = df1.copy()
    df_2 = df2.copy()
    cumulative_sum = df_1.groupby(cols)[target].cumsum() - df_1[target]
    cumulative_count = df_1.groupby(cols).cumcount()
    df_1[cols + "_mean_target"] = cumulative_sum/cumulative_count
    vals =df_1.groupby(cols).agg({target:['mean']})
    vals.columns = [x[0] for x in vals.columns]
    vals.rename(columns={target:cols+'_mean_target'},inplace=True)
    df_2 = pd.merge(df_2,vals,on=cols,how='left')
    df_1.fillna(df_1[cols + "_mean_target"].mean(),inplace=True)
    df_2.fillna(df_2[cols + "_mean_target"].mean(),inplace=True)
    df_1.drop([cols],axis=1,inplace=True)
    df_2.drop([cols],axis=1,inplace=True)
    return df_1,df_2,vals

In [1]:
X_train = df_train.drop(['ID'],axis=1)
y_train=df_train['Crop_Damage']
X_train,_,vals = expanding_mean(X_train,X_train,target='Crop_Damage',cols='Groups')df_test_id = df_test.copy()
df_test_id = df_test.copy()
df_test.drop('ID',axis=1,inplace=True)
df_test_id.reset_index(inplace=True)
X_train.drop('Crop_Damage',axis=1,inplace=True)
df_test = pd.merge(df_test,vals,on='Groups',how='left')

SyntaxError: invalid syntax (<ipython-input-1-cca2add83608>, line 3)

In [ ]:
splits = 15
folds =StratifiedKFold(n_splits=splits,random_state=22,shuffle=True)
# predictions = np.zeros((len(X_valid), 1))
oof_preds = np.zeros((len(df_test), 3))
feature_importance_df = pd.DataFrame()
feature_importance_df['Feature'] = X_train.columns
final_preds = []
random_state = [77,89,22,1007,1997,1890,2000,2020,8989,786,787,1999992,2021,7654]
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train,y_train)):
        print("Fold {}".format(fold_))
        
        X_trn,y_trn = X_train[cols].iloc[trn_idx],y_train.iloc[trn_idx]
        X_val,y_val = X_train[cols].iloc[val_idx],y_train.iloc[val_idx]
        clf = lgb.LGBMClassifier(random_state=22,n_jobs=-1,n_estimators=1000)
        clf.fit(X_trn, y_trn,eval_set=[(X_val,y_val)],eval_metric='multi_logloss',verbose=100, early_stopping_rounds=100)
        final_preds.append(accuracy_score(y_pred=clf.predict(X_val),y_true=y_val))
#         imp = importances(clf,X_val,y_val)
#         imp.rename(columns={
#             'Importance':f'Importance_{fold_}'
#         },inplace=True)
#         feature_importance_df = pd.merge(feature_importance_df,imp,on='Feature')
#         predictions += np.abs(clf.predict(X_valid)).reshape(-1,1)
        oof_preds += clf.predict_proba(df_test[cols])
print(sum(final_preds)/splits)
print(final_preds)

Finally i just took my oof predictions as my final submission probably a bad idea to use a stratified kfold, and just better go with a train test split , as i saw in my submissions because my public LB is 96.xx and Private is 95.84xx, just didnt got the time to experiment with that.

In [ ]:
df_sub = pd.read_csv('sample_submission.csv')

In [ ]:
df_sub['Crop_Damage'] = [np.argmax(x) for x in fin_preds]
df_sub['ID'] = df_test_id['ID']

In [ ]:
df_sub.to_csv('Final.csv',index=False)